In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/14 00:11:06 WARN Utils: Your hostname, NivethanAsus resolves to a loopback address: 127.0.1.1; using 172.20.150.32 instead (on interface eth0)
23/09/14 00:11:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/14 00:11:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/14 00:11:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/14 00:11:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
merchant_agg = spark.read.parquet("../../../data/insights/agg_insight_data/merchant_agg.parquet/")
consumers_agg = spark.read.parquet("../../../data/insights/agg_insight_data/consumers_agg.parquet/")
orders_agg = spark.read.parquet("../../../data/insights/agg_insight_data/orders_agg.parquet/")
postcode_agg = spark.read.parquet("../../../data/insights/agg_insight_data/postcode_agg.parquet/")
descriptions_agg = spark.read.parquet("../../../data/insights/agg_insight_data/descriptions_agg.parquet/")

In [4]:
print(merchant_agg.columns)

['merchant_abn', 'name', 'revenue_level', 'take_rate', 'average_merchant_fraud_probability']


In [5]:
print(consumers_agg.columns)

['merchant_abn', 'number_of_consumers', 'average_consumer_fraud_probability', 'number_of_repeat_consumers']


In [6]:
print(orders_agg.columns)

['merchant_abn', 'number_of_orders', 'average_cost_of_order']


In [7]:
print(postcode_agg.columns)

['merchant_abn', 'name', 'number_of_postcodes', 'avg_total_weekly_personal_income', 'avg_total_weekly_fam_income', 'avg_median_age', 'avg_num_of_consumers_per_postcode']


In [8]:
postcode_agg = postcode_agg.drop("name")

In [9]:
print(descriptions_agg.columns)

['merchant_abn', 'merchant_description']


In [10]:
joined_df = merchant_agg.join(consumers_agg, on = "merchant_abn", how = "leftouter").\
                join(orders_agg, on = "merchant_abn", how = "leftouter").\
                join(postcode_agg, on = "merchant_abn", how = "leftouter").\
                join(descriptions_agg, on = "merchant_abn", how = "leftouter")

In [11]:
joined_df.show()

+------------+--------------------+-------------+---------+----------------------------------+-------------------+----------------------------------+--------------------------+----------------+---------------------+-------------------+--------------------------------+---------------------------+------------------+---------------------------------+--------------------+
|merchant_abn|                name|revenue_level|take_rate|average_merchant_fraud_probability|number_of_consumers|average_consumer_fraud_probability|number_of_repeat_consumers|number_of_orders|average_cost_of_order|number_of_postcodes|avg_total_weekly_personal_income|avg_total_weekly_fam_income|    avg_median_age|avg_num_of_consumers_per_postcode|merchant_description|
+------------+--------------------+-------------+---------+----------------------------------+-------------------+----------------------------------+--------------------------+----------------+---------------------+-------------------+-----------------------

In [12]:
joined_df.write.mode("overwrite").parquet("../../../data/insights/joined.parquet")